# Importing Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

# import my tANS function
from Functions import Coder, Utils, CompTensor

In [2]:
range_ = (0,51)

# Importing The Symbol Table

In [3]:
s_tabs = [ pd.read_csv(f"trace/mobilenet_v2/weight_{i}_flat.apack", sep = " ", header = None) for i in range(range_[0],range_[1])]

for s_tab in s_tabs:            
    s_tab.columns = ["vmin","OL","abits","obits","vcnt"]

s_tabs[0]

,vmin,OL,abits,obits,vcnt
0,0,0,412,0,186
1,1,5,453,1365,273
2,32,4,144,120,30
3,48,4,69,44,11
4,64,4,43,24,6
5,80,4,30,16,4
6,96,4,18,8,2
7,112,4,43,24,6
8,128,4,30,16,4
9,144,4,10,4,1


# Importing the data

In [4]:
# importing the data
data = [np.load(f"trace/mobilenet_v2/weight_{i}.npy") for i in range(range_[0],range_[1])]
#data_flat = data.flatten()
np.unique(data[0]), data[0].flatten().shape

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  43,  44,  45,  46,  47,  48,  51,  52,  55,  56,  58,  59,
         61,  62,  64,  66,  67,  70,  82,  86,  94, 105, 112, 119, 127,
        128, 129, 153, 162, 173, 180, 181, 183, 187, 188, 189, 191, 192,
        193, 196, 197, 198, 199, 200, 204, 208, 209, 210, 211, 212, 213,
        214, 215, 216, 217, 219, 220, 221, 222, 223, 224, 225, 226, 227,
        228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240,
        241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
        254, 255], dtype=uint8),
 (864,))

# Compressing the data


1. Convert each data point to a *(symbol, offset)* pair
2. Compress Values to value stream
3. Put offset in offset stream

### *(Symbol, Offset)* Pair

In [5]:
# converting each data point to a symbol, offset pair

comp_tensors = []
for i, dat in enumerate(data):
    print(f"Processing data {i}")
    comp_tensors.append([CompTensor.CompTensor(dat.flatten(),s_tabs[i])])

# Access points of the first CompTensor of the first sublist
print(comp_tensors[0][0].points[0])

Processing data 0
Processing data 1
Processing data 2
Processing data 3
Processing data 4
Processing data 5
Processing data 6
Processing data 7
Processing data 8
Processing data 9
Processing data 10
Processing data 11
Processing data 12
Processing data 13
Processing data 14
Processing data 15
Processing data 16
Processing data 17
Processing data 18
Processing data 19
Processing data 20
Processing data 21
Processing data 22
Processing data 23
Processing data 24
Processing data 25
Processing data 26
Processing data 27
Processing data 28
Processing data 29
Processing data 30
Processing data 31
Processing data 32
Processing data 33
Processing data 34
Processing data 35
Processing data 36
Processing data 37
Processing data 38
Processing data 39
Processing data 40
Processing data 41
Processing data 42
Processing data 43
Processing data 44
Processing data 45
Processing data 46
Processing data 47
Processing data 48
Processing data 49
Processing data 50
Point: 1, Symbol: 1, Offset: 0


### Prep Symbols for Compression

In [6]:
# Getting freqs, must be a power of 2
freqs = []

for s_tab in s_tabs:
    # Get frequencies from the symbol table
    freq = list(s_tab.vcnt)

    # rescale so the sum of freq is 2**10, this ensures the Coder works effieciently
    # before I was just rescaling to the most accurate power of 2, but the coder would time out
    # building the object
    # Note: the rescale_list_to_power_of_2 function ensures that the sum of the list is a power of 2
    #       and also that no element is zero (bumps up the smallest elements to 1)
    freq = Utils.rescale_list_to_power_of_2(freq, 2**10)
    
    # append to freqs
    freqs.append(freq)

    print("Sum:", sum(freq))
    
print(freqs[1])

Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
Sum: 1024
[228, 306, 14, 14, 7, 7, 4, 1, 4, 1, 1, 1, 11, 21, 46, 358]


### OFfsets

In [7]:
def int_to_binary_list(value, nbits):
    if value >= 2**nbits or value < 0:
        raise ValueError(f"Value {value} cannot be represented in {nbits} bits.")
    
    binary_list = [int(bit) for bit in bin(value)[2:].zfill(nbits)]
    return binary_list

In [8]:
# make offset bitstream for one tensor

offset_stream = []

for i in range(len(comp_tensors)):
    offset_stream.append([])
    for j in range(len(comp_tensors[i])):
        offset_stream[i].append([])
        for k in range(len(comp_tensors[i][j].points)):
            offset_stream[i][j].extend(int_to_binary_list(comp_tensors[i][j].points[k].off, comp_tensors[i][j].points[k].OL))

In [9]:
# get shape of the offset stream
print(len(offset_stream), len(offset_stream[0]), len(offset_stream[0][0]), len(offset_stream[0][0])/8)

51 1 3171 396.375


In [10]:
import time
nbits = 8 # takes 4 bits to represent each symbol

all_run_times = []
all_build_times = []
all_comp_ratios = []
all_bps = []

for i in range(len(freqs)):
    print(f"Compressing Layer {i}")

    run_times = []
    build_times = []
    comp_ratios = []
    bp_sym = []

    for j in range(len(comp_tensors[i])):
        print(f"\tCompressing tensor {j}")
        
        # compressing the symbols
        time_start = time.time()
        
        c = Coder.Coder(sum(freqs[i]), [i for i in range(len(freqs[i]))], freqs[i], fast = False)
        
        time_end = time.time()
        build_time_taken = time_end - time_start

        msg = [p.symbol for p in comp_tensors[i][j].points]

        time_start = time.time()
        out, comp_bits = c.encode_decode(msg)
        time_end = time.time()
        run_time_taken = time_end - time_start
        
        # factoring in the offset bits  
        total_comp_bits = comp_bits + len(offset_stream[i][j])

        if out != msg:
            print("Error in encoding and decoding")
            break
        
        run_times.append(run_time_taken)
        build_times.append(build_time_taken)
        comp_ratios.append(len(msg) * nbits / total_comp_bits)
        bp_sym.append(total_comp_bits / len(msg))
        
    # print average stats
    print("\tAverage run time taken: %f seconds" % np.mean(run_times))
    print("\tAverage build time taken: %f seconds" % np.mean(build_times))
    print("\tAverage compression ratio: %f" % np.mean(comp_ratios))
    print("\tAverage bits per symbol: %f" % np.mean(bp_sym))
    
    # add stats to all lists
    all_run_times.append(run_times)
    all_build_times.append(build_times)
    all_comp_ratios.append(comp_ratios)
    all_bps.append(bp_sym)

Compressing Layer 0
	Compressing tensor 0
	Average run time taken: 0.005223 seconds
	Average build time taken: 0.035180 seconds
	Average compression ratio: 1.276454
	Average bits per symbol: 6.267361
Compressing Layer 1
	Compressing tensor 0
	Average run time taken: 0.001328 seconds
	Average build time taken: 0.009823 seconds
	Average compression ratio: 1.343440
	Average bits per symbol: 5.954861
Compressing Layer 2
	Compressing tensor 0
	Average run time taken: 0.002599 seconds
	Average build time taken: 0.009485 seconds
	Average compression ratio: 1.181766
	Average bits per symbol: 6.769531
Compressing Layer 3
	Compressing tensor 0
	Average run time taken: 0.010384 seconds
	Average build time taken: 0.009316 seconds
	Average compression ratio: 1.210163
	Average bits per symbol: 6.610677
Compressing Layer 4
	Compressing tensor 0
	Average run time taken: 0.004640 seconds
	Average build time taken: 0.009287 seconds
	Average compression ratio: 1.220554
	Average bits per symbol: 6.554398


KeyboardInterrupt: 

In [ ]:
# display stats in a dataframe
import pandas as pd

freqs = freqs

stats_1 = pd.DataFrame({"Layer": [i for i in range(len(freqs))],
                      "Run Time": [np.mean(all_run_times[i]) for i in range(len(freqs))],
                      "Build Time": [np.mean(all_build_times[i]) for i in range(len(freqs))],
                      "Compression Ratio": [np.mean(all_comp_ratios[i]) for i in range(len(freqs))],
                      "Bits per Symbol": [np.mean(all_bps[i]) for i in range(len(freqs))]})

In [ ]:
stats_1

,Layer,Run Time,Build Time,Compression Ratio,Bits per Symbol
0,0,0.004729,0.035203,1.276454,6.267361
1,1,0.001214,0.009375,1.343440,5.954861
2,2,0.002544,0.009440,1.181766,6.769531
3,3,0.010095,0.009288,1.210163,6.610677
4,4,0.004659,0.009494,1.220554,6.554398
5,5,0.019341,0.009295,1.128720,7.087674
6,6,0.038031,0.009370,1.178215,6.789931
7,7,0.008323,0.010224,1.183562,6.759259
8,8,0.036721,0.009595,1.207442,6.625579
9,9,0.036175,0.009600,1.214123,6.589120


In [ ]:
# get average compressino ration
print("Average Compression Ratio:", np.mean(stats_1["Compression Ratio"]))

Average Compression Ratio: 1.2613272803064175


In [ ]:
# olde ratio: 1.2613272803064175